# imports

In [1]:
# all imports

# buffer
from collections import defaultdict
import random
from abc import ABC, abstractmethod
from typing import (
    Any,
    Dict,
    Generic,
    Optional,
    List,
    TYPE_CHECKING,
    Set,
    TypeVar,
)

import torch
from numpy import inf
from torch import cat, Tensor
from torch.nn import Module
from torch.utils.data import DataLoader

from avalanche.benchmarks.utils import (
    classification_subset,
    AvalancheDataset,
)
from avalanche.models import FeatureExtractorBackbone
# from ..benchmarks.utils.utils import concat_datasets
from avalanche.benchmarks.utils import concat_datasets
from avalanche.training.storage_policy import ReservoirSamplingBuffer, BalancedExemplarsBuffer, ClassBalancedBuffer
# from avalanche.training.storage_policy
if TYPE_CHECKING:
    from .templates import SupervisedTemplate, BaseSGDTemplate

from avalanche.training.storage_policy import ParametricBuffer, RandomExemplarsSelectionStrategy
from avalanche.benchmarks.utils.data_loader import ReplayDataLoader
from avalanche.training.plugins import SupervisedPlugin
from typing import Optional, TYPE_CHECKING

from avalanche.benchmarks.utils import concat_classification_datasets
from avalanche.training.plugins.strategy_plugin import SupervisedPlugin
from avalanche.training.storage_policy import (
    ExemplarsBuffer,
    ExperienceBalancedBuffer,
)

if TYPE_CHECKING:
    from avalanche.training.templates import SupervisedTemplate


# dataset
from avalanche.benchmarks import SplitMNIST, SplitCIFAR100
from avalanche.benchmarks.utils.data_loader import GroupBalancedDataLoader, ReplayDataLoader

from torch.utils.data.dataset import Subset, ConcatDataset, TensorDataset


import torch
import matplotlib.pyplot as plt
import numpy as np
from torch.nn import CrossEntropyLoss
from torch.optim import SGD

import torch.optim.lr_scheduler # ?

from avalanche.benchmarks.classic import SplitCIFAR100
from avalanche.benchmarks.classic import SplitCIFAR10

from avalanche.benchmarks.generators import nc_benchmark, ni_benchmark

from avalanche.benchmarks.generators import filelist_benchmark, dataset_benchmark, \
                                            tensors_benchmark, paths_benchmark

from torchvision import transforms
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, CenterCrop
from torchvision.transforms.functional import center_crop

from torchvision import transforms
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, RandomHorizontalFlip, Resize
import os

from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.logging import InteractiveLogger, TensorboardLogger, \
    WandBLogger, TextLogger

from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, loss_metrics

from avalanche.training.plugins.checkpoint import CheckpointPlugin, \
    FileSystemCheckpointStorage
from avalanche.training.determinism.rng_manager import RNGManager

# training
from avalanche.training import Naive, CWRStar, Replay, GDumb, \
    Cumulative, LwF, GEM, AGEM, EWC, AR1

# strategies
from avalanche.models import SimpleMLP
from torch.optim import SGD
from torch.nn import CrossEntropyLoss

from torchvision.models import resnet18, ResNet18_Weights

# training
from avalanche.training import Naive, CWRStar, Replay, GDumb, \
    Cumulative, LwF, GEM, AGEM, EWC, AR1

# strategies
from avalanche.models import SimpleMLP
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.training.plugins import ReplayPlugin

from types import SimpleNamespace

from avalanche.training.storage_policy import ParametricBuffer, RandomExemplarsSelectionStrategy


In [2]:
torch.cuda.set_device(3)

In [3]:
if torch.cuda.is_available():
    current_gpu = torch.cuda.current_device()
    print(f"Current default GPU index: {current_gpu}")
    print(f"Current default GPU name: {torch.cuda.get_device_name(current_gpu)}")
else:
    print("No GPUs available.")

Current default GPU index: 3
Current default GPU name: NVIDIA A40


# Replay Plugin

In [4]:
# from avalanche.benchmarks.utils.data_loader import ReplayDataLoader
# from avalanche.training.plugins import SupervisedPlugin

# class CustomReplay(SupervisedPlugin):
#     def __init__(
#         self,
#         mem_size: int = 200,
#         batch_size: Optional[int] = None,
#         batch_size_mem: Optional[int] = None,
#         task_balanced_dataloader: bool = False,
#         storage_policy: Optional["ExemplarsBuffer"] = None,
#         # The policy that controls how to add new exemplars in memory
#                         #
#     ):
#         super().__init__()
#         self.mem_size = mem_size
#         self.batch_size = batch_size
#         self.batch_size_mem = batch_size_mem
#         self.task_balanced_dataloader = task_balanced_dataloader

#         self.storage_policy = storage_policy
#         assert storage_policy.max_size == self.mem_size

#     def before_training_exp(self,
#                             strategy: "SupervisedTemplate",
#                             num_workers: int = 0,
#                             shuffle: bool = True,
#                             **kwargs):
#         """ Here we set the dataloader. create batch with examples from both
#         training data and external memory"""
#         if len(self.storage_policy.buffer) == 0:
#             # first experience. We don't use the buffer, no need to change
#             # the dataloader.
#             return

#         # replay dataloader samples mini-batches from the memory and current
#         # data separately and combines them together.
#         print("Override the dataloader.")
#         strategy.dataloader = ReplayDataLoader(
#             strategy.adapted_dataset,
#             self.storage_policy.buffer,
#             oversample_small_tasks=True,
#             num_workers=num_workers,
#             batch_size=strategy.train_mb_size,
#             shuffle=shuffle)

#     def after_training_exp(self, strategy: "SupervisedTemplate", **kwargs):
#         """ We update the buffer after the experience.
#             You can use a different callback to update the buffer in a different place
#         """
#         print("Buffer update.")
#         self.storage_policy.update(strategy, **kwargs)

from avalanche.benchmarks.utils.data_loader import ReplayDataLoader
from avalanche.training.plugins import SupervisedPlugin
from typing import Optional, TYPE_CHECKING

from avalanche.benchmarks.utils import concat_classification_datasets
from avalanche.benchmarks.utils.data_loader import ReplayDataLoader
from avalanche.training.plugins.strategy_plugin import SupervisedPlugin
from avalanche.training.storage_policy import (
    ExemplarsBuffer,
    ExperienceBalancedBuffer,
)

if TYPE_CHECKING:
    from avalanche.training.templates import SupervisedTemplate

import random
def sample_Avadataset(dataset, percentage= 0.1):
    """random sample in certain percentage, percentage should be float"""
#     random.seed(41)
    torch.manual_seed(41)
    data_len = len(dataset)
#     indices = list(range(data_len))
    indices = torch.randperm(data_len).tolist()
#     random.shuffle(indices)
    sample_num = int(data_len * percentage)
#     sampled_indices = random.sample(indices, sample_num)
#     new_buffer_data = dataset.subset(sampled_indices)
    sampled_indices = indices[:sample_num]  # Select the first 'sample_num' indices
    new_buffer_data = dataset.subset(sampled_indices)

    return new_buffer_data



class CustomReplay(SupervisedPlugin):
    def __init__(
        self,
        mem_size: int = 200,
        batch_size: Optional[int] = None,
        batch_size_mem: Optional[int] = None,
        task_balanced_dataloader: bool = False,
        storage_policy: Optional["ExemplarsBuffer"] = None,
        # The policy that controls how to add new exemplars in memory
                        #
    ):
        super().__init__()
        self.mem_size = mem_size
        self.batch_size = batch_size
        self.batch_size_mem = batch_size_mem
        self.task_balanced_dataloader = task_balanced_dataloader

        self.storage_policy = storage_policy
        assert storage_policy.max_size == self.mem_size



    def before_training_exp(self,
                            strategy: "SupervisedTemplate",
                            num_workers: int = 0,
                            shuffle: bool = True,
                            **kwargs):
        """ Here we set the dataloader. create batch with examples from both
        training data and external memory"""
        if len(self.storage_policy.buffer) == 0:
            return

        # replay dataloader samples mini-batches from the memory and current
        # data separately and combines them together.
        print("Override the dataloader.")

        buffer_size = len(self.storage_policy.buffer)
        print("buffer size: " + str(buffer_size))
        num_class = len(self.storage_policy.buffer_datasets)
        print("current class number in replay buffer: " + str(num_class))
        # for item in self.storage_policy.buffer:
        # tempory_buffer = AvalancheDataset([])

        tempory_buffer = []

        for i, dataset in enumerate(self.storage_policy.buffer_datasets):
            sub_data = sample_Avadataset(dataset)
            if i == 0:
                tempory_buffer = sub_data
            else:
                tempory_buffer = tempory_buffer.concat(sub_data)


        # buffer = [sample_Avadataset(dataset) for dataset in self.storage_policy.buffer_datasets]
        strategy.dataloader = ReplayDataLoader(
            strategy.adapted_dataset,
            # self.storage_policy.buffer,
            tempory_buffer,
            num_workers=num_workers,
            batch_size=strategy.train_mb_size,
            shuffle=shuffle)
        # for x, y, t in dl:
        #     print(t.tolist())
        #     print(len(t.tolist()))

    def after_training_exp(self, strategy: "SupervisedTemplate", **kwargs):
        """ We update the buffer after the experience.
            You can use a different callback to update the buffer in a different place
        """
        print("Buffer update.")
        self.storage_policy.update(strategy, **kwargs)


In [5]:
# self.storage_policy.update(strategy, **kwargs)-> parametric buffer update

In [6]:
from torch.utils.data.dataset import Subset, ConcatDataset, TensorDataset

class NewParametricBuffer(BalancedExemplarsBuffer):
    """Stores samples for replay using a custom selection strategy and
    grouping."""

    def __init__(
        self,
        max_size: int,
        groupby=None,
        selection_strategy: Optional["ExemplarsSelectionStrategy"] = None,
    ):
        """Init.
        :param max_size: The max capacity of the replay memory.
        :param groupby: Grouping mechanism. One of {None, 'class', 'task',
            'experience'}.
        :param selection_strategy: The strategy used to select exemplars to
            keep in memory when cutting it off.
        """
        super().__init__(max_size)
        assert groupby in {None, "task", "class", "experience"}, (
            "Unknown grouping scheme. Must be one of {None, 'task', "
            "'class', 'experience'}"
        )
        self.groupby = groupby
        ss = selection_strategy or RandomExemplarsSelectionStrategy()
        self.selection_strategy = ss
        self.seen_groups: Set[int] = set()
        self._curr_strategy = None


    def update(self, strategy: "SupervisedTemplate", **kwargs):
        assert strategy.experience is not None
        new_data: AvalancheDataset = strategy.experience.dataset
        new_groups = self._make_groups(strategy, new_data)
        self.seen_groups.update(new_groups.keys())

        # associate lengths to classes
        lens = self.get_group_lengths(len(self.seen_groups))
        group_to_len = {}
        for group_id, ll in zip(self.seen_groups, lens):
            group_to_len[group_id] = ll

        # update buffers with new data
        for group_id, new_data_g in new_groups.items():
            ll = group_to_len[group_id]
            if group_id in self.buffer_groups:
                old_buffer_g = self.buffer_groups[group_id]
                old_buffer_g.update_from_dataset(strategy, new_data_g)
                old_buffer_g.resize(strategy, ll)
            else:
                new_buffer = _ParametricSingleBuffer(ll, self.selection_strategy)
                new_buffer.update_from_dataset(strategy, new_data_g)
                self.buffer_groups[group_id] = new_buffer

        # resize buffers
        for group_id, class_buf in self.buffer_groups.items():
            self.buffer_groups[group_id].resize(strategy, group_to_len[group_id])

    def _make_groups(
        self, strategy, data: AvalancheDataset
    ) -> Dict[int, AvalancheDataset]:
        """Split the data by group according to `self.groupby`."""
        # if self.groupby is None:
        #     return {0: data}
        # elif self.groupby == "task":
        #     return self._split_by_task(data)
        # elif self.groupby == "experience":
        #     return self._split_by_experience(strategy, data)
        # elif self.groupby == "class":
        return self._split_by_class(data)
        # else:
        #     assert False, "Invalid groupby key. Should never get here."

    def _split_by_class(self, data: AvalancheDataset) -> Dict[int, AvalancheDataset]:
        # Get sample idxs per class
        cl_idxs: Dict[int, List[int]] = defaultdict(list)
        targets = getattr(data, "targets")
        for idx, target in enumerate(targets):
            target = int(target)
            cl_idxs[target].append(idx)

        # Make AvalancheSubset per class
        new_groups: Dict[int, AvalancheDataset] = {}
        for c, c_idxs in cl_idxs.items():
            new_groups[c] = classification_subset(data, indices=c_idxs)
        return new_groups

    # def _split_by_experience(
    #     self, strategy, data: AvalancheDataset
    # ) -> Dict[int, AvalancheDataset]:
    #     exp_id = strategy.clock.train_exp_counter + 1
    #     return {exp_id: data}

    # def _split_by_task(self, data: AvalancheDataset) -> Dict[int, AvalancheDataset]:
    #     new_groups = {}
    #     task_set = getattr(data, "task_set")
    #     for task_id in task_set:
    #         new_groups[task_id] = task_set[task_id]
    #     return new_groups



class _ParametricSingleBuffer(ExemplarsBuffer):
    """A buffer that stores samples for replay using a custom selection
    strategy.

    This is a private class. Use `ParametricBalancedBuffer` with
    `groupby=None` to get the same behavior.
    """

    def __init__(
        self,
        max_size: int,
        selection_strategy: Optional["ExemplarsSelectionStrategy"] = None,
    ):
        """
        :param max_size: The max capacity of the replay memory.
        :param selection_strategy: The strategy used to select exemplars to
                                   keep in memory when cutting it off.
        """
        super().__init__(max_size)
        ss = selection_strategy or RandomExemplarsSelectionStrategy()
        self.selection_strategy = ss
        self._curr_strategy = None

    def update(self, strategy: "SupervisedTemplate", **kwargs):
        assert strategy.experience is not None
        new_data = strategy.experience.dataset
        self.update_from_dataset(strategy, new_data)

    def update_from_dataset(self, strategy, new_data):
        # print(new_data[1])
        data_len = len(new_data)
        indices = list(range(data_len))
        random.shuffle(indices)
        sample_num = int(data_len * 0.2)
        sampled_indices = random.sample(indices, sample_num)
        # new_buffer_data = new_data[:sample_num]
        # new_buffer_data = Subset(new_data, sampled_indices)
        new_buffer_data = new_data.subset(sampled_indices)

        self.buffer = self.buffer.concat(new_buffer_data)
        self.resize(strategy, self.max_size)

    def resize(self, strategy, new_size: int):
        self.max_size = new_size
        idxs = self.selection_strategy.make_sorted_indices(
            strategy=strategy, data=self.buffer
        )
        self.buffer = self.buffer.subset(idxs[: self.max_size])


In [7]:
# def data
#     total_indices = len(data)
#     num_samples = int(total_indices * self.percentage)  # Calculate number of samples based on percentage

#     indices = list(range(total_indices))  # Generate a list of all indices
#     random.shuffle(indices)

#     sampled_indices = random.sample(indices, num_samples)  # Randomly sample indices

#     return sampled_indices

In [8]:
# class ParametricBuffer(BalancedExemplarsBuffer):
#     def __init__(self, max_size: int, percent_samples_per_class: float, groupby='class', selection_strategy: Optional["ExemplarsSelectionStrategy"] = None):
#         """
#         :param percent_samples_per_class: The percentage of samples to keep from each class after an update.
#         """
#         super().__init__(max_size, groupby, selection_strategy)
#         self.percent_samples_per_class = percent_samples_per_class

#     def update(self, strategy: "SupervisedTemplate", **kwargs):
#         super().update(strategy, **kwargs)  # Assuming super().update does the necessary updates
#         self._adjust_buffer_to_percent()

#     def _adjust_buffer_to_percent(self):
#         """
#         Adjusts the buffer to retain only a specified percentage of examples from each class.
#         """
#         for group_id, buffer_group in self.buffer_groups.items():
#             num_samples_to_keep = int(len(buffer_group) * self.percent_samples_per_class)
#             if len(buffer_group) > num_samples_to_keep:
#                 selected_indices = np.random.choice(len(buffer_group), num_samples_to_keep, replace=False)
#                 self.buffer_groups[group_id] = buffer_group.subset(selected_indices)


In [9]:
l = [1,2,3,4]
random.sample(l, 2)

[2, 4]

# Data transform

In [10]:
# !pip install pytorch_pretrained_vit


import torch
import matplotlib.pyplot as plt
import numpy as np
from torch.nn import CrossEntropyLoss
from torch.optim import SGD

import torch.optim.lr_scheduler # ?

from avalanche.benchmarks.classic import SplitCIFAR100
from avalanche.benchmarks.classic import SplitCIFAR10

from avalanche.benchmarks.generators import nc_benchmark, ni_benchmark

from avalanche.benchmarks.generators import filelist_benchmark, dataset_benchmark, \
                                            tensors_benchmark, paths_benchmark

from torchvision import transforms
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, CenterCrop
from torchvision.transforms.functional import center_crop

In [57]:
from torchvision import transforms
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, RandomHorizontalFlip, Resize
import os
# stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
# stats = ((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
stats = ((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

transform_train = Compose([
    Resize((224, 224)),
    # Resize(384),
    RandomHorizontalFlip(),
    ToTensor(),
    Normalize(*stats,inplace=True)
])

transform_test = Compose([
    Resize((224, 224)),
    # Resize(384),
    ToTensor(),
    Normalize(*stats,inplace=True)
])

# from torchvision.transforms import Resize
# transform1 = Compose([
#     Resize(224),
#     ToTensor()
# ])

In [47]:
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.logging import InteractiveLogger, TensorboardLogger, \
    WandBLogger, TextLogger
# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('../logs/method2_10buffer_imagenet.txt', 'w'))

# print to stdout
interactive_logger = InteractiveLogger()

In [48]:
# evaluation
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, loss_metrics, class_accuracy_metrics

# The evaluation plugin manages the metrics computation.
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=False, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=False, epoch=True, experience=True, stream=True),
        class_accuracy_metrics(minibatch=False, epoch=False, epoch_running=False, experience=False, stream=True),
    # forgetting_metrics(experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger])

In [49]:
from avalanche.training.plugins.checkpoint import CheckpointPlugin, \
    FileSystemCheckpointStorage
from avalanche.training.determinism.rng_manager import RNGManager


In [50]:
RNGManager.set_random_seeds(1234)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

checkpoint_plugin = CheckpointPlugin(
    FileSystemCheckpointStorage(
        directory='./checkpoints/task_cifar',
    ),
    map_location=device
)

# Load checkpoint (if exists in the given storage)
# If it does not exist, strategy will be None and initial_exp will be 0
strategy, initial_exp = checkpoint_plugin.load_checkpoint_if_exists()

cuda


In [51]:
# training
from avalanche.training import Naive, CWRStar, Replay, GDumb, \
    Cumulative, LwF, GEM, AGEM, EWC, AR1

# strategies
from avalanche.models import SimpleMLP
from torch.optim import SGD
from torch.nn import CrossEntropyLoss

## data, benchmark prepare

In [22]:
import json
import os

def load_json(file_path):
    """
    Load a JSON file and return the data.

    :param file_path: Path to the JSON file.
    :return: Parsed JSON data.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

In [26]:
label_path = '../saved_data/ImageNet/imagenet_new_labels_to_names.json'
integer_to_name = load_json(label_path)
integer_to_name = {int(label): name for (label, name) in integer_to_name.items()}

In [27]:
print(integer_to_name)

{0: 'chambered nautilus, pearly nautilus, nautilus', 1: 'harvestman, daddy longlegs, Phalangium opilio', 2: 'macaw', 3: 'bittern', 4: 'electric ray, crampfish, numbfish, torpedo', 5: 'drake', 6: 'agama', 7: 'night snake, Hypsiglena torquata', 8: 'indigo bunting, indigo finch, indigo bird, Passerina cyanea', 9: 'tiger shark, Galeocerdo cuvieri', 10: 'flamingo', 11: 'garter snake, grass snake', 12: 'common newt, Triturus vulgaris', 13: 'tench, Tinca tinca', 14: 'sulphur-crested cockatoo, Kakatoe galerita, Cacatua galerita', 15: 'white stork, Ciconia ciconia', 16: 'terrapin', 17: 'diamondback, diamondback rattlesnake, Crotalus adamanteus', 18: 'flatworm, platyhelminth', 19: 'sea snake', 20: 'spiny lobster, langouste, rock lobster, crawfish, crayfish, sea crawfish', 21: 'green snake, grass snake', 22: 'bald eagle, American eagle, Haliaeetus leucocephalus', 23: 'kite', 24: 'goldfish, Carassius auratus', 25: 'scorpion', 26: 'goose', 27: 'bulbul', 28: 'chickadee', 29: 'sea anemone, anemone', 

In [30]:
# sythnthesis classes
benchmark_cifar = SplitCIFAR100(n_experiences=20,
                          seed = 41,             
                          )

orders = benchmark_cifar.classes_order
order_list = [orders[x:x+5] for x in range(0, len(orders), 5)]
print(order_list)
# order_sample = [order[3:] for order in order_list]
# classname_list = []
# label_list = []
# classname_list_sep = []
# for order_l in order_sample:
#     label_list.append(order_l)
#     cur_classname = [integer_to_name[i] for i in order_l]
#     classname_list.append(cur_classname)
# classname_list_sep = [item for lists in classname_list for item in lists]
# label_list_sep = [item for lists in label_list for item in lists]
# print(label_list_sep)

Files already downloaded and verified
Files already downloaded and verified
[[36, 0, 54, 5, 20], [22, 45, 13, 83, 19], [26, 73, 16, 62, 33], [34, 98, 24, 74, 53], [10, 94, 51, 4, 32], [38, 81, 50, 40, 41], [30, 89, 69, 64, 21], [84, 14, 88, 49, 68], [6, 80, 57, 65, 46], [9, 91, 48, 72, 31], [76, 7, 47, 8, 1], [61, 75, 63, 18, 86], [59, 70, 43, 85, 95], [27, 93, 35, 25, 82], [44, 56, 67, 66, 37], [60, 11, 2, 78, 52], [97, 39, 55, 3, 99], [29, 71, 23, 28, 90], [87, 15, 92, 17, 77], [12, 42, 96, 79, 58]]


In [34]:
def combine_files_with_numbers(folder, file_initial, numbers, output_folder):
    """use to get the data with label in the training experience"""
    combined_content = ""  # Initialize an empty string to store combined content
    # Compile a set of filenames to look for, based on the list of numbers
    filenames_to_look_for = {file_initial + f"{number}.txt" for number in numbers}
    print(filenames_to_look_for)
    
    os.makedirs(output_folder, exist_ok=True)

    # Iterate over each file in the specified folder
#     files_found = 0
    
    for file in os.listdir(folder):
        # Check if the file name matches exactly any in our set of filenames to look for
        if file in filenames_to_look_for:
#             print(f'found file {file}')
            # Open and read the file, then add its content to the combined_content string
            with open(os.path.join(folder, file), 'r') as f:
                combined_content += f.read()  # Add a newline character after each file's content for better separation
#                 print(combined_content)

    joined_string = '_'.join(str(integer) for integer in numbers)


    output_file_path = os.path.join(output_folder, f"{file_initial}combined_{joined_string}.txt")
    print(output_file_path)
    with open(output_file_path, 'w') as f:
        f.write(combined_content)

In [35]:
image_folder = '../saved_data/ImageNet/ImageNet_train'
output_folder = image_folder + '_combined/'

train_experience_list = []
for l in order_list:
    combine_files_with_numbers(
                image_folder,
                'class',
                l,
                output_folder)
    
    joined_string = '_'.join(str(integer) for integer in l)
    file_initial = 'class'
    output_file_path = output_folder +file_initial+ 'combined' + '_' + joined_string + '.txt' 
    train_experience_list.append(output_file_path)

{'class5.txt', 'class0.txt', 'class54.txt', 'class36.txt', 'class20.txt'}
../saved_data/ImageNet/ImageNet_train_combined/classcombined_36_0_54_5_20.txt
{'class22.txt', 'class19.txt', 'class45.txt', 'class13.txt', 'class83.txt'}
../saved_data/ImageNet/ImageNet_train_combined/classcombined_22_45_13_83_19.txt
{'class26.txt', 'class16.txt', 'class62.txt', 'class73.txt', 'class33.txt'}
../saved_data/ImageNet/ImageNet_train_combined/classcombined_26_73_16_62_33.txt
{'class98.txt', 'class34.txt', 'class74.txt', 'class24.txt', 'class53.txt'}
../saved_data/ImageNet/ImageNet_train_combined/classcombined_34_98_24_74_53.txt
{'class10.txt', 'class4.txt', 'class94.txt', 'class32.txt', 'class51.txt'}
../saved_data/ImageNet/ImageNet_train_combined/classcombined_10_94_51_4_32.txt
{'class81.txt', 'class50.txt', 'class38.txt', 'class41.txt', 'class40.txt'}
../saved_data/ImageNet/ImageNet_train_combined/classcombined_38_81_50_40_41.txt
{'class69.txt', 'class64.txt', 'class89.txt', 'class30.txt', 'class21.

In [36]:
image_folder = '../saved_data/ImageNet/ImageNet_val'
output_folder = image_folder + '_combined/'

test_experience_list = []
for l in order_list:
    combine_files_with_numbers(
                image_folder,
                'class',
                l,
                output_folder)
    
    joined_string = '_'.join(str(integer) for integer in l)
    file_initial = 'class'
    output_file_path = output_folder +file_initial+ 'combined' + '_' + joined_string + '.txt' 
    test_experience_list.append(output_file_path)


{'class5.txt', 'class0.txt', 'class54.txt', 'class36.txt', 'class20.txt'}
../saved_data/ImageNet/ImageNet_val_combined/classcombined_36_0_54_5_20.txt
{'class22.txt', 'class19.txt', 'class45.txt', 'class13.txt', 'class83.txt'}
../saved_data/ImageNet/ImageNet_val_combined/classcombined_22_45_13_83_19.txt
{'class26.txt', 'class16.txt', 'class62.txt', 'class73.txt', 'class33.txt'}
../saved_data/ImageNet/ImageNet_val_combined/classcombined_26_73_16_62_33.txt
{'class98.txt', 'class34.txt', 'class74.txt', 'class24.txt', 'class53.txt'}
../saved_data/ImageNet/ImageNet_val_combined/classcombined_34_98_24_74_53.txt
{'class10.txt', 'class4.txt', 'class94.txt', 'class32.txt', 'class51.txt'}
../saved_data/ImageNet/ImageNet_val_combined/classcombined_10_94_51_4_32.txt
{'class81.txt', 'class50.txt', 'class38.txt', 'class41.txt', 'class40.txt'}
../saved_data/ImageNet/ImageNet_val_combined/classcombined_38_81_50_40_41.txt
{'class69.txt', 'class64.txt', 'class89.txt', 'class30.txt', 'class21.txt'}
../sav

In [58]:
benchmark =  filelist_benchmark(
                                None,
                                train_file_lists = train_experience_list, # train
                                test_file_lists = test_experience_list, # test
                                task_labels = [0]*20,
                                # complete_test_set_only=True,
                                train_transform=transform_train,
                            eval_transform=transform_train
                            )

  0%|                                                                       | 0/204 [01:41<?, ?it/s]


In [59]:
train_stream = benchmark.train_stream
for experience in train_stream:
    t = experience.task_label
    exp_id = experience.current_experience
    training_dataset = experience.dataset
    print('Task {} batch {} -> train'.format(t, exp_id))
    print('This batch contains', len(training_dataset), 'patterns')
    print(f'This contains labels {experience.classes_in_this_experience}')

Task 0 batch 0 -> train
This batch contains 6500 patterns
This contains labels [0, 36, 5, 20, 54]
Task 0 batch 1 -> train
This batch contains 6500 patterns
This contains labels [13, 45, 19, 83, 22]
Task 0 batch 2 -> train
This batch contains 6500 patterns
This contains labels [33, 73, 16, 26, 62]
Task 0 batch 3 -> train
This batch contains 6500 patterns
This contains labels [34, 98, 74, 53, 24]
Task 0 batch 4 -> train
This batch contains 6500 patterns
This contains labels [32, 4, 10, 51, 94]
Task 0 batch 5 -> train
This batch contains 6500 patterns
This contains labels [38, 40, 41, 81, 50]
Task 0 batch 6 -> train
This batch contains 6500 patterns
This contains labels [64, 69, 21, 89, 30]
Task 0 batch 7 -> train
This batch contains 6500 patterns
This contains labels [68, 14, 49, 84, 88]
Task 0 batch 8 -> train
This batch contains 6500 patterns
This contains labels [65, 6, 46, 80, 57]
Task 0 batch 9 -> train
This batch contains 6500 patterns
This contains labels [72, 9, 48, 91, 31]
Task 

## experiment

In [60]:
from torchvision.models import resnet18, ResNet18_Weights
weights = ResNet18_Weights.DEFAULT
resnet_model = resnet18(weights=weights)

In [61]:
# training
from avalanche.training import Naive, CWRStar, Replay, GDumb, \
    Cumulative, LwF, GEM, AGEM, EWC, AR1

# strategies
from avalanche.models import SimpleMLP
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.training.plugins import ReplayPlugin

from types import SimpleNamespace

In [62]:
storage_p = ParametricBuffer(
    max_size=60000,
    groupby='class',
    selection_strategy=RandomExemplarsSelectionStrategy()
    # selection_strategy=RandomExemplarsSelectionStrategy()

)


In [63]:
cl_strategy = Naive(
    resnet_model, torch.optim.SGD(resnet_model.parameters(), lr=0.01, momentum = 0.9),
    CrossEntropyLoss(), train_mb_size=32, train_epochs=50, eval_mb_size=16,
    # eval_every=500,
    device=device,
    evaluator=eval_plugin,
    plugins=[CustomReplay(mem_size=60000, storage_policy = storage_p)]
    )
# mem_size: int = 200,
        # batch_size: Optional[int] = None,
        # batch_size_mem: Optional[int] = None,
        # task_balanced_dataloader: bool = False,
        # storage_policy: Optional["ExemplarsBuffer"] = None,

## 10% memory

In [64]:
# Training
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(benchmark.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 36, 5, 20, 54]
-- >> Start of training phase << --
  7%|████▎                                                         | 14/204 [01:10<05:35,  1.77s/it]

KeyboardInterrupt: 

In [ ]:
results

## 20% memory

In [38]:
# Training
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(benchmark.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 36, 5, 20, 54]
-- >> Start of training phase << --
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:05<00:00, 14.28it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.3095
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.7148
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:04<00:00, 19.20it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4378
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8676
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

In [ ]:
def plot_losses(losses1, label1='Run 1 Loss'):
    """
    Plots the training losses from two experimental runs.

    Parameters:
    - losses1: List of losses from the first run.
    - losses2: List of losses from the second run.
    - label1: Label for the first run.
    - label2: Label for the second run.
    """
    plt.figure(figsize=(5, 4))
    plt.plot(losses1, label=label1, marker='o')
#     plt.title(title)
    plt.xlabel('# tasks')
    plt.ylabel('Accuarcy')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
plot_losses(results, label1='replay_20\% original data')